In [1]:
import inspect
import os
import time
from pathlib import Path

import gymnasium
from gymnasium import Env
from gymnasium.vector import VectorEnv

from sac import init_action_selector, init_policy, init_optimizer, wrap_env, policy_construction_hyper_parameter
from src.datetime import get_current_timestamp
from src.experiment_logging.experiment_logger import ExperimentLogger, log_experiment
from src.model_db.model_db import ModelDB
from src.model_db.dummy_model_db import DummyModelDB
from src.np_functions import inv_symmetric_log
from src.reinforcement_learning.algorithms.policy_mitosis.mitosis_policy_info import MitosisPolicyInfo
from src.model_db.tiny_model_db import TinyModelDB
from src.module_analysis import count_parameters, get_gradients_per_parameter
from src.moving_averages import ExponentialMovingAverage
from src.reinforcement_learning.core.action_selectors.action_selector import ActionSelector
from src.reinforcement_learning.core.action_selectors.predicted_std_action_selector import PredictedStdActionSelector
from src.reinforcement_learning.core.action_selectors.state_dependent_noise_action_selector import \
    StateDependentNoiseActionSelector
from src.reinforcement_learning.core.generalized_advantage_estimate import compute_episode_returns, compute_returns
from src.reinforcement_learning.core.policies.base_policy import BasePolicy
from src.reinforcement_learning.core.policy_construction import InitActionSelectorFunction, PolicyConstruction
from src.reinforcement_learning.gym.envs.test_env import TestEnv
from src.schedulers import FixedValueScheduler, OneStepRecursiveScheduler
from src.stopwatch import Stopwatch
from src.summary_statistics import format_summary_statics
from src.reinforcement_learning.core.policies.actor_critic_policy import ActorCriticPolicy
from typing import Any, SupportsFloat, Optional
from gymnasium.wrappers import RecordVideo, AutoResetWrapper, NormalizeReward, TransformReward, TransformObservation, ClipAction
from src.reinforcement_learning.core.callback import Callback
from src.reinforcement_learning.algorithms.sac.sac import SAC, SAC_DEFAULT_OPTIMIZER_PROVIDER
from src.reinforcement_learning.algorithms.ppo.ppo import PPO, PPOLoggingConfig
from src.reinforcement_learning.core.normalization import NormalizationType
from src.torch_device import set_default_torch_device, optimizer_to_device
from src.reinforcement_learning.gym.parallelize_env import parallelize_env_async
from torch.distributions import Normal, Categorical

import torch
from torch import optim, nn
import torch.distributions as dist
import gymnasium as gym
import numpy as np

from src.torch_functions import antisymmetric_power

%load_ext autoreload
%autoreload 2

pygame 2.5.2 (SDL 2.28.3, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
from src.summary_statistics import maybe_compute_summary_statistics
from src.reinforcement_learning.core.loss_config import LossLoggingConfig
from src.reinforcement_learning.algorithms.sac.sac import SAC, SACLoggingConfig
def get_setup() -> dict[str, str]:
    import inspect
    import sac
    return {
        'sac.py': inspect.getsource(sac),
        'notebook': _ih[1] + '\n\n' + _ih[-1], # first and last cell input (imports and this cell)
    }

policy_id: str
policy: BasePolicy
optimizer: optim.Optimizer
wrapped_env: Env
steps_trained: int
def get_policy(create_new_if_exists: bool):
    
    global policy_id, policy, optimizer, wrapped_env, steps_trained
    
    policy_in_ram = 'policy' in globals()
    if not policy_in_ram or create_new_if_exists:
        if not policy_in_ram:
            print('No policy in RAM, creating a new one')
        
        policy_id = get_current_timestamp()
        policy, optimizer, wrapped_env = PolicyConstruction.init_from_info(
            env=env,
            info=PolicyConstruction.create_policy_initialization_info(
                init_action_selector=init_action_selector,
                init_policy=init_policy,
                init_optimizer=init_optimizer,
                wrap_env=wrap_env,
                hyper_parameters=policy_construction_hyper_parameter,
            ),
        )
        steps_trained = 0
        print(f'New policy {policy_id} created')
    
    if parent_policy_id is not None:
        model_entry = policy_db.load_model_state_dict(policy, parent_policy_id)
        steps_trained = model_entry['model_info']['steps_trained']
        print(f'Loading state dict from policy {parent_policy_id}')
    
    print(f'Using policy {policy_id} with parent policy {parent_policy_id}')
    return policy_id, policy, optimizer, wrapped_env, steps_trained

score_mean_ema = ExponentialMovingAverage(alpha=0.25)
step_stopwatch = Stopwatch()
total_stopwatch = Stopwatch()
best_iteration_score = -1e6

def on_rollout_done(rl: SAC, step: int, info: dict[str, Any], scheduler_values: dict[str, Any]):
    
    if step % 1000 != 0:
        return
    
    tail_indices = rl.buffer.tail_indices(1000)
    
    # rewards = rl.buffer.rewards[tail_indices]
    # if 'raw_rewards' in info['rollout']:
    #     rewards = info['rollout']['raw_rewards']
    
    # episode_scores = compute_episode_returns(
    #     rewards=rewards,
    #     episode_starts=np.repeat(np.arange(len(tail_indices)).reshape(-1, 1), num_envs, axis=1) % 1000 == 0,
    #     last_episode_starts=info['last_episode_starts'],
    #     gamma=1.0,
    #     gae_lambda=1.0,
    #     normalize_rewards=None,
    #     remove_unfinished_episodes=True,
    # )
    
    episode_scores = rl.buffer.compute_most_recent_episode_scores(rl.num_envs)
    
    if len(episode_scores) > 0:
    
        global best_iteration_score
        iteration_score = episode_scores.mean()
        score_moving_average = score_mean_ema.update(iteration_score)
        if iteration_score >= best_iteration_score:
            best_iteration_score = iteration_score
            policy_db.save_model_state_dict(
                model_id=policy_id,
                parent_model_id=parent_policy_id,
                model_info={
                    'score': iteration_score.item(),
                    'steps_trained': steps_trained,
                    'wrap_env_source_code': wrap_env_source_code_source,
                    'init_policy_source_code': init_policy_source
                },
                model=policy,
                optimizer=optimizer,
            )
        info['score_moving_average'] = score_moving_average
    
    info['episode_scores'] = episode_scores
        
def on_optimization_done(rl: SAC, step: int, info: dict[str, Any], scheduler_values: dict[str, Any]):
    # global steps_trained
    # steps_trained += rl.buffer.pos
    
    if step % 1000 != 0:
        return
    num_env_steps = step * rl.num_envs
    
    step_time = step_stopwatch.reset()
    total_time = total_stopwatch.time_passed()
    
    tail_indices = rl.buffer.tail_indices(1000)
    
    episode_scores = info.get('episode_scores')
    score_moving_average = info.get('score_moving_average') or 0.0
    
    scores = format_summary_statics(
        episode_scores, 
        mean_format=' 6.3f',
        std_format='4.3f',
        min_value_format=' 6.3f',
        max_value_format='5.3f',
        n_format='>2'
    )
    # scores2 = format_summary_statics(
    #     rl.buffer.compute_most_recent_episode_scores(rl.num_envs, lambda r: 1 * r), 
    #     mean_format=' 6.3f',
    #     std_format='4.3f',
    #     min_value_format=' 6.3f',
    #     max_value_format='5.3f',
    #     n_format='>2'
    # )
    # advantages = format_summary_statics(
    #     rl.buffer.advantages, 
    #     mean_format=' 6.3f',
    #     std_format='.1f',
    #     min_value_format=' 7.3f',
    #     max_value_format='6.3f',
    # )
    actor_loss = format_summary_statics(
        info['final_actor_loss'],  
        mean_format=' 5.3f',
        # std_format='5.3f',
        std_format=None,
        min_value_format=None,
        max_value_format=None,
    )
    # actor_loss_raw = format_summary_statics(
    #     info['raw_actor_loss'],  
    #     mean_format=' 5.3f',
    #     std_format='5.3f',
    #     min_value_format=None,
    #     max_value_format=None,
    # )
    entropy_coef_loss = None if 'final_entropy_coef_loss' not in info else format_summary_statics(
        info['final_entropy_coef_loss'], 
        mean_format='5.3f',
#         std_format='5.3f',
        std_format=None,
        min_value_format=None,
        max_value_format=None,
    )
    critic_loss = format_summary_statics(
        info['final_critic_loss'], 
        mean_format='5.3f',
#         std_format='5.3f',
        std_format=None,
        min_value_format=None,
        max_value_format=None,
    )
    entropy_coef = format_summary_statics(
        info['entropy_coef'],
        mean_format='5.3f',
#         std_format='5.3f',
        std_format=None,
        min_value_format=None,
        max_value_format=None,
    )
    # resets = format_summary_statics(
    #     rl.buffer.dones.astype(int).sum(axis=0), 
    #     mean_format='.2f',
    #     std_format=None,
    #     min_value_format='1d',
    #     max_value_format=None,
    # )
    # kl_div = info['actor_kl_divergence'][-1]
    # grad_norm = format_summary_statics(
    #     info['grad_norm'], 
    #     mean_format=' 6.3f',
    #     std_format='.1f',
    #     min_value_format=' 7.3f',
    #     max_value_format='6.3f',
    # )
    action_stds = info['rollout'].get('action_stds')
    if action_stds is not None:
        rollout_action_stds = format_summary_statics(
            action_stds,
            mean_format='5.3f',
            std_format='5.3f',
            min_value_format=None,
            max_value_format=None,
        )
    else:
        rollout_action_stds = 'N/A'
    action_magnitude = format_summary_statics(
        np.abs(rl.buffer.actions[tail_indices]),
        mean_format='5.3f',
        std_format='5.3f',
        min_value_format=None,
        max_value_format=None,
    )
    # ppo_epochs = info['nr_ppo_epochs']
    # ppo_updates = info['nr_ppo_updates']
    # expl_var = rl.buffer.compute_critic_explained_variance()
    print(f"{step = : >7}, "
          f"{num_env_steps = : >7}, "
          f"{scores = :s}, "
          # f"{scores2 = :s}, "
          f'score_ema = {score_moving_average: 6.3f}, '
          # f"{advantages = :s}, "
          f"{actor_loss = :s}, "
          # f"{actor_loss_raw = :s}, "
          +(f"{entropy_coef_loss = :s}, " if entropy_coef_loss is not None else '')+
          f"{critic_loss = :s}, "
          f"{entropy_coef = :s}, "
          f"rollout_stds = {rollout_action_stds:s}, "
          f"{action_magnitude = :s}, "
          # f"{expl_var = :.3f}, "
          # f"{kl_div = :.4f}, "
          # f"{ppo_epochs = }, "
          # f"{ppo_updates = }, "
          # f"{grad_norm = :s}, "
          f"n_updates = {rl.gradient_steps_performed}, "
          # f"{resets = :s}, "
          f"time = {step_time:4.1f}, "
          f"total_time = {total_time:4.1f} \n"
          )
    logger.add_item({
        'step': step,
        'num_env_steps': num_env_steps,
        'scores': maybe_compute_summary_statistics(episode_scores),
        'actor_loss': maybe_compute_summary_statistics(info['final_actor_loss']),
        'entropy_coef_loss': maybe_compute_summary_statistics(info.get('final_entropy_coef_loss')),
        'critic_loss': maybe_compute_summary_statistics(info['final_critic_loss']),
        'entropy_coef': maybe_compute_summary_statistics(info['entropy_coef']),
        'action_stds': maybe_compute_summary_statistics(action_stds),
        'action_magnitude': maybe_compute_summary_statistics(np.abs(rl.buffer.actions[tail_indices])),
        'num_gradient_steps': rl.gradient_steps_performed,
        'step_time': step_time,
        'total_time': total_time
    })
    if step % 10000 == 0:
        logger.save_experiment_log()
        print()
    print()
    
    # if episode_scores is not None and len(episode_scores) > 0 and episode_scores.mean().item() < -500:
    #     logger.save_experiment_log()
    #     raise ValueError('Score too low, policy probably fucked :(')

device = torch.device("cuda:0") if True else torch.device('cpu')
print(f'using device {device}')

def create_env(render_mode: str | None):
    return gym.make(env_name, render_mode=render_mode, **env_kwargs)

wrap_env_source_code_source = inspect.getsource(wrap_env)
init_policy_source = inspect.getsource(init_policy)

env_name = 'HalfCheetah-v4'
# env_kwargs = {'forward_reward_weight': 1.25, 'healthy_reward': 0.5, 'ctrl_cost_weight': 0.001 }
# env_kwargs = {'forward_reward_weight': 1.25, 'ctrl_cost_weight': 0.1 }
# env_kwargs = {'forward_reward_weight': 1.25, 'ctrl_cost_weight': 0.05 }
env_kwargs = {}
num_envs = 1
    
# policy_db = TinyModelDB[MitosisPolicyInfo](base_path=f'saved_models/rl/{env_name}')
policy_db = DummyModelDB[MitosisPolicyInfo]()
print(f'{policy_db = }')

parent_policy_id=None  # '2024-04-28_20.57.23'

# TODO
# env = parallelize_env_async(lambda: create_env(render_mode=None), num_envs)
env = create_env(render_mode=None)

logger = ExperimentLogger(f'experiment_logs/{env_name}/sac/')

try:
    policy_id, policy, optimizer, wrapped_env, steps_trained = get_policy(create_new_if_exists=False)
    print(f'{count_parameters(policy) = }')
    print(f'{env = }, {num_envs = }')
        
    with ((torch.autograd.set_detect_anomaly(False))):
        algo = SAC(
            env=wrapped_env,
            policy=policy,
            actor_optimizer_provider=lambda params: optim.Adam(params, lr=3e-4),  # (params, lr=3e-4, betas=(0.5, 0.999)),
            critic_optimizer_provider=lambda params: optim.Adam(params, lr=3e-4),  # (params, lr=3e-4, betas=(0.5, 0.999)),
            # weigh_and_reduce_actor_loss=lambda l: 1 * l.mean(),
            # weigh_critic_loss=lambda l: 1 * l,
            buffer_size=1_000_000,
            reward_scale=1,
            gamma=0.99,
            tau=0.005,
            entropy_coef_optimizer_provider=lambda params: optim.Adam(params, lr=3e-4),
            entropy_coef=1.0,
            rollout_steps=1,
            gradient_steps=1,
            warmup_steps=10_000,
            optimization_batch_size=256,
            target_update_interval=1,
            # sde_noise_sample_freq=50,
            callback=Callback(
                on_rollout_done=on_rollout_done,
                rollout_schedulers={},
                on_optimization_done=on_optimization_done,
                optimization_schedulers={},
            ),
            logging_config=SACLoggingConfig(log_rollout_infos=True, log_rollout_action_stds=True,
                                            log_last_obs=True, log_entropy_coef=True,
                                            entropy_coef_loss=LossLoggingConfig(log_final=True),
                                            actor_loss=LossLoggingConfig(log_final=True, log_raw=True),
                                            critic_loss=LossLoggingConfig(log_final=True)),
            torch_device=device,
        )
        total_stopwatch.reset()
        with log_experiment(
            logger,
            experiment_tags=algo.collect_tags(),
            hyper_parameters=algo.collect_hyper_parameters(),
            setup=get_setup(),
        ) as x:
            logger.save_experiment_log()
            print('\nStarting Training\n\n')
            # import cProfile
            # pr = cProfile.Profile()
            # pr.enable()
            algo.learn(5_000_000)
            # pr.disable()  
            # pr.dump_stats('profile_stats.pstat')
except KeyboardInterrupt:
    print('keyboard interrupt')
finally:
    print('closing envs')
    time.sleep(0.5)
    env.close()
    print('envs closed')
    policy_db.close()
    print('model db closed')
    

print('done')

using device cuda:0
policy_db = DummyModelDB()
No policy in RAM, creating a new one
New policy 2024-10-10 19:40:29.409681 created
Using policy 2024-10-10 19:40:29.409681 with parent policy None
count_parameters(policy) = 217870
env = <TimeLimit<OrderEnforcing<PassiveEnvChecker<HalfCheetahEnv<HalfCheetah-v4>>>>>, num_envs = 1
Grabbing system information... done!
saved experiment log 2024-10-10_19-40-29_893791~93ggDv at experiment_logs/HalfCheetah-v4/sac/2024-10-10_19-40-29_893791~93ggDv.json

Starting Training

step =   11000, num_env_steps =   11000, scores = -240.794 (n= 1), score_ema = -257.082, actor_loss = -18.099, entropy_coef_loss = -3.001, critic_loss = 1.201, entropy_coef = 0.741, rollout_stds = 0.920 ± 0.036, action_magnitude = 0.523 ± 0.289, n_updates = 1000, time = 17.4, total_time = 16.9 

step =   12000, num_env_steps =   12000, scores = -186.816 (n= 1), score_ema = -239.516, actor_loss = -26.110, entropy_coef_loss = -5.989, critic_loss = 1.214, entropy_coef = 0.549, rollo